# Whale and Dolphin Classification Project

Authors:
- Victor Möslein
- Maren Rieker
- Reed Garvin
- Dinah Rabe

This Notebook is one of three core notebooks of the Whale and Dolphin Classification Project for the "Machine Learning" class at the Hertie School. It focuses on the application of classic machine learning models to the task at hand. There is one other notebook concerned with data preprocessing and another that focuses on the application of a deep learning model.

The code of this notebook partly follows the chapter on Classification from the book "Hands-on Machine Learning with Scikit-Learn, Keras, and Tensorflow" by Aurélien Géron.

In [ ]:
## Setup: System settings and packages

In [1]:
# Python ≥3.5 is required

import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "1.0"

import random
import numpy as np
from numpy import load
import pandas as pd
from numpy import savez_compressed
import os
import timeit
import seaborn as sns
import pickle
import PIL


# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# to make this notebook's output stable
np.random.seed(42)

In [ ]:
full_data_switch_on = True # if the full data set should be used, this switch need to be set to true

## Define paths to data and for output

In [ ]:
# path to clean data folder
ROOT_PATH_DATA = "input/04_cleaned/"

# where to save figures
ROOT_PATH_FIG = "output/ml_models/01_figures"
os.makedirs(ROOT_PATH_FIG, exist_ok=True)

# where to save output

ROOT_OUTPUT = "output/ml_models/"
OUTPUT_PATH_TRAIN_EVAL = os.path.join(ROOT_OUTPUT + "02_training_set_evaluation")
OUTPUT_PATH_TEST_EVAL = os.path.join(ROOT_OUTPUT + "03_test_set_evaluation")
OUTPUT_PATH_HYPPAR_TUN = os.path.join(ROOT_OUTPUT + "04_hyperparamter_tuning")
OUTPUT_PATH_RUN_TIME = os.path.join(ROOT_OUTPUT + "05_runtime_stats")

# function to save figures

def save_fig(fig_id, save_path=ROOT_PATH_FIG, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(save_path, fig_id + "." + fig_extension)
    print(">... Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Loading and splitting training data

In [ ]:
labels_df = pd.read_csv(ROOT_PATH_DATA + "train/clean_train.csv", sep = ';')

In [ ]:
labels_df.head() # Checking for correct dimensions after every step. The data set contains 51033 images.

In [ ]:
labels_df.shape

In [ ]:
labels_full = labels_df["species"]

In [ ]:
labels_full.head()

In [ ]:
labels_full.shape

In [ ]:
pic_ids_full = labels_df["image"]

In [ ]:
pic_ids_full.head()

In [ ]:
pic_ids_full.shape

In [ ]:
# load npz files ## has to be adjusted to the actual one
img_data = np.load("input/04_cleaned/train/img_data_full_48.npz")
img_data_full = img_data["arr_0"]

In [ ]:
img_data_full.shape

In [ ]:
# Split into training and test set - 10.000 test set / 40.000 full training set
# stratify = labels splits it proportionally to classes in the dataset

from sklearn.model_selection import train_test_split
img_data_train_full, img_data_test_no_PCA, labels_train_full, labels_test_no_PCA, pic_ids_train_full, pic_ids_test_no_PCA = train_test_split(img_data_full , labels_full, pic_ids_full, stratify=labels_full, test_size=0.2, random_state=42)


In [ ]:
# Split into training and validation set - 30.000 training set / 10.000 validation set
img_data_train_no_PCA, img_data_val_no_PCA, labels_train_no_PCA, labels_val_no_PCA, pic_ids_train_no_PCA, pic_ids_val_no_PCA = train_test_split(img_data_train_full , labels_train_full, pic_ids_train_full, train_size=30000, random_state=42)

## Speed improvements through dimensionality reduction


### Performing Principal Component Analysis on the training data
#### this had to be moved to the beginning of the project due to computation time restrictions

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
img_data_full.shape

In [ ]:
# defining to keep 99% of the variance of the data
pca = PCA(.95)
img_data_full_red = pca.fit_transform(img_data_full)

In [ ]:
# checking how many features are left
img_data_full_red.shape

### 1466 features are left when keeping 99% of variance (still too many)
### 502 features are left when keeping 95% of variance

In [ ]:
# save dataset after PCA

def store_dict_red(image_df, target_pixel):
    if full_data_switch_on == True:
        savez_compressed("input/04_cleaned/train/img_data_full_red_"+str(target_pixel)+".npz",image_df)
        print("file successfully stored in: input/04_cleaned/train/img_data_full_red_"+str(target_pixel)+".npz")
    elif full_data_switch_on == False:
        savez_compressed("input/04_cleaned/train/img_data_sample_red_"+str(target_pixel)+".npz",image_df)
        print("file successfully stored in: input/04_cleaned/train/img_data_sample_red_"+str(target_pixel)+".npz")
    else:
        raise ValueError("Full data switch is not correctly defined")

### Store reduced image file

In [ ]:
store_dict_red(img_data_full_red, target_pixel=48)

### Load reduced image file

In [ ]:
# load npz files
img_data_red = np.load("input/04_cleaned/train/img_data_full_red_48.npz")
img_data_full_red = img_data_red["arr_0"]

In [ ]:
# Split into training and test set - 10.000 test set / 40.000 full training set
# stratify = labels splits it proportionally to classes in the dataset

from sklearn.model_selection import train_test_split
img_data_train_full, img_data_test, labels_train_full, labels_test, pic_ids_train_full, pic_ids_test = train_test_split(img_data_full_red , labels_full, pic_ids_full, stratify=labels_full, test_size=0.2, random_state=42)


In [ ]:
# Split into training and validation set - 30.000 training set / 10.000 validation set
img_data_train, img_data_val, labels_train, labels_val, pic_ids_train, pic_ids_val = train_test_split(img_data_train_full , labels_train_full, pic_ids_train_full, train_size=30000, random_state=42)


In [ ]:
# Checking for correct dimensions.

In [ ]:
img_data_train_full.shape

In [ ]:
img_data_test.shape

In [ ]:
img_data_train.shape

In [ ]:
img_data_val.shape

In [ ]:
# The distribution between training and validation set is similar.
labels_val.value_counts()

In [ ]:
labels_train_full.value_counts()

In [ ]:
# So now we have: 30000 images in the training set, 10826 images in the validation set, 10207 images in the test set

## Implementing baseline model

In [ ]:
def train_clasf(classifier_x, img_data_train, labels_train):
    # set name of classifier
    classifier_name = classifier_x.__class__.__name__

    # train model
    print(">... Starting training of", classifier_name)
    start_time = timeit.default_timer()
    classifier_x.fit(img_data_train, labels_train)
    time_elapsed = timeit.default_timer() - start_time

    print(">... Classifier {} sucessfully trained in {} seconds.".format(classifier_name, round(time_elapsed,3)))


In [ ]:
# Running this on the Hertie Server takes around 2 hours with max_iter = 500 and 24 minutes with max_iter = 100 on full image data
# After PCA, it runs in 96 seconds locally with max_iter = 500

# We set solver = 'sag' because stochastic average gradient descent works better on large data sets

from sklearn.linear_model import LogisticRegression

classifier_LR = LogisticRegression(random_state=42, max_iter = 500, solver = 'sag')
train_clasf(classifier_LR, img_data_train, labels_train)

## Evaluating baseline model ("compute metrics on train AND dev")

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
# store predictions of classifier
pred_train = classifier_LR.predict(img_data_train)
pred_val = classifier_LR.predict(img_data_val)

# evaluate classifier and store metrics
evaluation_scores = {"Precision Score Train": precision_score(labels_train, pred_train, average = "macro").round(3),
                     "Precision Score Validation": precision_score(labels_val, pred_val, average = "macro").round(3),
                     "Recall Score Train": recall_score(labels_train, pred_train, average = "macro").round(3),
                     "Recall Score Validation": recall_score(labels_val, pred_val, average = "macro").round(3),
                     "F1 Score Train": f1_score(labels_train, pred_train, average="macro").round(3),
                     "F1 Score Validation": f1_score(labels_val, pred_val, average="macro").round(3)}


In [ ]:
# save evaluation scores
def store_eval_score(image_df, classifier_name):
    savez_compressed(OUTPUT_PATH_TRAIN_EVAL + "/evaluation_scores_"+str(classifier_name)+".npz",image_df)
    print("file successfully stored in: output/ml_models/02_training_set_evaluation")


In [ ]:
store_eval_score(evaluation_scores, "LogisticRegression")

In [ ]:
## Multiclass Confusion Matrix

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

conf_matrix = multilabel_confusion_matrix(labels_val,pred_val,labels=labels_val)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

disp_conf_matrix = ConfusionMatrixDisplay.from_predictions(labels_val, pred_val)

disp_conf_matrix.show()

In [ ]:
plt.show()

In [ ]:
# def function for saving the filtered error table

def store_conf_matrix(disp_conf_matrix, classifier_name):
    disp_conf_matrix.savefig(OUTPUT_PATH_TRAIN_EVAL + "/conf_matrix_"+str(classifier_name)+".jpg")
    print("file successfully stored in: output/ml_models/02_training_set_evaluation")

In [ ]:
# saving the confusion matrix
store_conf_matrix(disp_conf_matrix, "LogisticRegression")

In [ ]:
# Inspecting the errors
output_dict = {}
output_array = np.c_[pic_ids_val, labels_val, pred_val] ## adjust name of pic_ids and labels depending on train or val

# Create error array with specific error
err_type_arr = np.array([])
for i in range(len(output_array)):
    if output_array[i,1] != output_array[i,2]:
        err_type_arr = np.append(err_type_arr, "error")
    else:
        err_type_arr = np.append(err_type_arr, "No error")

error_table_pd = pd.DataFrame(output_array)
error_table_pd.rename(columns = {0:'Picture ID', 1:"Label", 2:"Predicted"}, inplace = True)
error_table_pd["Error Check"] = err_type_arr

# print filtered error table
print(error_table_pd.loc[error_table_pd["Error Check"].isin(["error"])].sort_values(by=["Label", "Picture ID"]))

In [ ]:
# def function for saving the filtered error table

def store_error_table(image_df, classifier_name):
    savez_compressed(OUTPUT_PATH_TRAIN_EVAL + "/error_table_"+str(classifier_name)+".npz",image_df)
    print("file successfully stored in: output/ml_models/02_training_set_evaluation")


In [ ]:
# transform pd frame into dictionary for saving
output_dict["error_table"] = error_table_pd

# saving the error table
store_error_table(output_dict, "LogisticRegression")

## Implementing RandomForest Classifier as advanced model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
num_trees = 500

# this works in 5 minutes locally after PCA

classifier_RF = RandomForestClassifier(max_depth = 10, n_estimators = num_trees, n_jobs=-1,random_state=42)
train_clasf(classifier_RF, img_data_train, labels_train)

## Evaluating RandomForest Classifier

In [ ]:
# store predictions of classifier ## Victor: ich war mir nicht ganz sicher ob das so richtig ist
pred_train = classifier_RF.predict(img_data_train)
pred_val = classifier_RF.predict(img_data_val)

In [ ]:
# evaluate classifier and store metrics
evaluation_scores = {"Precision Score Train": precision_score(labels_train, pred_train, average = "macro").round(3),
                     "Precision Score Validation": precision_score(labels_val, pred_val, average = "macro").round(3),
                     "Recall Score Train": recall_score(labels_train, pred_train, average = "macro").round(3),
                     "Recall Score Validation": recall_score(labels_val, pred_val, average = "macro").round(3),
                     "F1 Score Train": f1_score(labels_train, pred_train, average="macro").round(3),
                     "F1 Score Validation": f1_score(labels_val, pred_val, average="macro").round(3)}

In [ ]:
store_eval_score(evaluation_scores, "RandomForest")

In [ ]:
# Inspecting the errors
output_dict = {}
output_array = np.c_[pic_ids_val, labels_val, pred_val] ## adjust name of pic_ids and labels depending on train or val

# Create error array with specific error
err_type_arr = np.array([])
for i in range(len(output_array)):
    if output_array[i,1] != output_array[i,2]:
        err_type_arr = np.append(err_type_arr, "error")
    else:
        err_type_arr = np.append(err_type_arr, "No error")

error_table_pd = pd.DataFrame(output_array)
error_table_pd.rename(columns = {0:'Picture ID', 1:"Label", 2:"Predicted"}, inplace = True)
error_table_pd["Error Check"] = err_type_arr

# print filtered error table
print(error_table_pd.loc[error_table_pd["Error Check"].isin(["error"])].sort_values(by=["Label", "Picture ID"]))

In [ ]:
# transform pd frame into dictionary for saving
output_dict["error_table"] = error_table_pd

# saving the error table
store_error_table(output_dict, "RandomForest")

In [ ]:
## Multiclass Confusion Matrix

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt

cm = confusion_matrix(labels_val, pred_val)
cm = confusion_matrix(np.arange(26), np.arange(26))
cmp = ConfusionMatrixDisplay(cm, display_labels=np.arange(26))
fig, ax = plt.subplots(figsize=(10,10))

In [ ]:
cmp.plot(ax=ax)

In [ ]:
disp = ConfusionMatrixDisplay.from_predictions(labels_val, pred_val)

In [ ]:
disp = disp.plt()

In [ ]:
# saving the confusion matrix
store_conf_matrix(fig, "RandomForest")

## Hyperparamter Tuning RandomForest Classifier

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
# Selecting Parameters of Random Forest Classifier for Grid Search

params_grid_RF = [
    {"n_estimators": [100, 500, 1000, 2000],
     "criterion":["gini","entropy"],
     "max_features": ["auto", "log2", "sqrt"],
     "bootstrap": [True, False],
     "max_depth": [90, 100, 110],
     "min_samples_leaf": [1, 2, 4],
     "min_samples_split": [2, 5, 10]}
]


In [ ]:
classifier_RF_hyper = RandomForestClassifier()

In [ ]:
# Perform Randomized Search

scoring = ["f1_macro", "accuracy", "precision_macro", "recall_macro"]


randomized_search = RandomizedSearchCV(estimator = classifier_RF_hyper,
                           param_distributions = params_grid_RF,
                           n_iter=10,
                           cv = 3,
                           scoring= scoring,
                           refit = "precision_macro",
                           n_jobs = -1,
                           verbose = 3,
                           return_train_score = True)

randomized_search.fit(img_data_train, labels_train)



In [ ]:
# Perform Grid-Search
#Takes too loooong

scoring = ["f1_macro", "precision_macro", "recall_macro"]


grid_search = GridSearchCV(estimator = classifier_RF_hyper,
                           param_grid = params_grid_RF,
                           cv = 3,
                           scoring= scoring,
                           refit = "precision_macro",
                           n_jobs = -1,
                           verbose = 3,
                           return_train_score = True)

grid_search.fit(img_data_train, labels_train)

In [ ]:
print(randomized_search.best_params_)
best_hyppar_dict = {}
best_hyppar_dict["Randomized_Search_Best_Params"] = randomized_search.best_params_

In [ ]:
def store_best_hyperpar(image_df, classifier_name):
    savez_compressed(OUTPUT_PATH_HYPPAR_TUN + "/best_hyperpar_"+str(classifier_name)+".npz",image_df)
    print("file successfully stored in: output/ml_models/04_hyperparamter_tuning")

In [ ]:
best_params_dict = best_hyppar_dict
store_best_hyperpar(best_params_dict, "RandomForestTuned")

## Evaluate tuned RF classifier

In [ ]:
## define the tuned classifier with the best hyperparamters
## these here are still random ones

num_trees = 10 # change to actual value

classifier_RF_tuned = RandomForestClassifier(class_weight='balanced',
                                      criterion='gini',
                                      max_depth=55,
                                      max_features='log2',
                                      min_samples_leaf=0.005,
                                      min_samples_split=0.005,
                                      n_estimators=num_trees)

classifier_RF_tuned.fit(img_data_train,labels_train)

In [ ]:
# store predictions of classifier
pred_train = classifier_RF_tuned.predict(img_data_train)
pred_val = classifier_RF_tuned.predict(img_data_val)

# evaluate classifier and store metrics
evaluation_scores = {"Precision Score Train": precision_score(labels_train, pred_train, average = "macro").round(3),
                     "Precision Score Validation": precision_score(labels_val, pred_val, average = "macro").round(3),
                     "Recall Score Train": recall_score(labels_train, pred_train, average = "macro").round(3),
                     "Recall Score Validation": recall_score(labels_val, pred_val, average = "macro").round(3),
                     "F1 Score Train": f1_score(labels_train, pred_train, average="macro").round(3),
                     "F1 Score Validation": f1_score(labels_val, pred_val, average="macro").round(3)}


In [ ]:
store_eval_score(evaluation_scores, "RandomForest_tuned")

In [ ]:
## Multiclass Confusion Matrix

### Checking Feature importance of RF

In [ ]:
# This needs to be done on the full image data (pre-PCA)
## define the tuned classifier with the best hyperparamters
## these here are still random ones

num_trees = 10 # change to actual value

classifier_RF_tuned_no_PCA = RandomForestClassifier(class_weight='balanced',
                                      criterion='gini',
                                      max_depth=55,
                                      max_features='log2',
                                      min_samples_leaf=0.005,
                                      min_samples_split=0.005,
                                      n_estimators=num_trees)

classifier_RF_tuned.fit(img_data_train_no_PCA,labels_train_no_PCA)

In [ ]:
# Defining the function to plot the digits of feature importance
# Adapted from Aurelien Geron:

pix_res = 48

def plot_digit(data):
    image = data.reshape(pix_res, pix_res)
    plt.imshow(image, cmap = mpl.cm.hot,
               interpolation="nearest")
    plt.axis("off")

In [ ]:
# Adding feature importances from 3 RGB values to one pixel
feature_imp_sum = np.empty([(int(len(classifier_RF_tuned_no_PCA.feature_importances_)/3)),])

for itr in range(int(len(classifier_RF_tuned_no_PCA.feature_importances_)/3)):
    r = int(itr*3)
    g = int(r+1)
    b = int(g+1)
    feature_imp_sum[itr] = classifier_RF_tuned_no_PCA.feature_importances_[r] + classifier_RF_tuned_no_PCA.feature_importances_[g] + classifier_RF_tuned_no_PCA.feature_importances_[b]


In [ ]:
# Plotting feature importance sum for every pixel to a plot and save it
plot_digit(feature_imp_sum)

cbar = plt.colorbar(ticks=[feature_imp_sum.min(), feature_imp_sum.max()])
cbar.ax.set_yticklabels(['Not important', 'Very important'])

In [ ]:
save_fig("RandomForest_feature_importance_plot_full_data")



## Implementing XGBoost as second advanced model

In [ ]:
import xgboost as xgb

classifier_XGB = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="auc")
classifier_XGB.fit(img_data_train, labels_train, early_stopping_rounds=5, eval_set=[(img_data_val, labels_val)])


## Evaluating XGBoost Classifier

In [ ]:
# store predictions of classifier
pred_train = classifier_XGB.predict(img_data_train)
pred_val = classifier_XGB.predict(img_data_val)

In [ ]:
# evaluate classifier and store metrics
evaluation_scores = {"Precision Score Train": precision_score(labels_train, pred_train, average = "macro").round(3),
                     "Precision Score Validation": precision_score(labels_val, pred_val, average = "macro").round(3),
                     "Recall Score Train": recall_score(labels_train, pred_train, average = "macro").round(3),
                     "Recall Score Validation": recall_score(labels_val, pred_val, average = "macro").round(3),
                     "F1 Score Train": f1_score(labels_train, pred_train, average="macro").round(3),
                     "F1 Score Validation": f1_score(labels_val, pred_val, average="macro").round(3)}

In [ ]:
store_eval_score(evaluation_scores, "XGBoost")

In [ ]:
# Inspecting the errors
output_dict = {}
output_array = np.c_[pic_ids_val, labels_val, pred_val] ## adjust name of pic_ids and labels depending on train or val

# Create error array with specific error
err_type_arr = np.array([])
for i in range(len(output_array)):
    if output_array[i,1] != output_array[i,2]:
        err_type_arr = np.append(err_type_arr, "error")
    else:
        err_type_arr = np.append(err_type_arr, "No error")

error_table_pd = pd.DataFrame(output_array)
error_table_pd.rename(columns = {0:'Picture ID', 1:"Label", 2:"Predicted"}, inplace = True)
error_table_pd["Error Check"] = err_type_arr

# print filtered error table
print(error_table_pd.loc[error_table_pd["Error Check"].isin(["error"])].sort_values(by=["Label", "Picture ID"]))

In [ ]:
# transform pd frame into dictionary for saving
output_dict["error_table"] = error_table_pd

# saving the error table
store_error_table(output_dict, "XGBoost")


## Hyperparamter Tuning XGBoost Classifier

In [ ]:
from scipy.stats import uniform

In [ ]:
# Selecting Parameters of XGBoost Classifier for Grid Search

params_grid_XGB = [{
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1
    "max_depth": [15, 20 25],
    "n_estimators": [400, 700, 1000],
    "subsample": uniform(0.6, 0.4)}
]


In [ ]:
classifier_XGB_hyper = xgb.XGBClassifier()

In [ ]:
# Perform Grid-Search

scoring = ["f1_macro", "precision_macro", "recall_macro"]

grid_search = GridSearchCV(estimator = classifier_XGB_hyper,
                           param_grid = params_grid_XGB,
                           cv = 3,
                           scoring= scoring,
                           refit = "precision",
                           n_jobs = -1,
                           verbose = 3,
                           return_train_score = True)

grid_search.fit(img_data_train, labels_train)

In [ ]:
print(grid_search.best_params_)
best_hyppar_dict = {}
best_hyppar_dict["Grid_Search_Best_Params"] = grid_search.best_params_


In [ ]:
best_params_dict = best_hyppar_dict
store_best_hyperpar(best_params_dict, "XGBoostTuned")

## Evaluate tuned XGBoost classifier

In [ ]:
## define the tuned classifier with the best hyperparamters
## these here are still random ones

classifier_XGB_tuned = xgb.XGBClassifier(colsample_bytree=0.7, gamma=0.5, learning_rate=0.3,max_depth=25, subsample=0.5,n_estimators=700)

classifier_XGB_tuned.fit(img_data_train,labels_train)

In [ ]:
# store predictions of classifier
pred_train = classifier_XGB_tuned.predict(img_data_train)
pred_val = classifier_XGB_tuned.predict(img_data_val)

# evaluate classifier and store metrics
evaluation_scores = {"Precision Score Train": precision_score(labels_train, pred_train, average = "macro").round(3),
                     "Precision Score Validation": precision_score(labels_val, pred_val, average = "macro").round(3),
                     "Recall Score Train": recall_score(labels_train, pred_train, average = "macro").round(3),
                     "Recall Score Validation": recall_score(labels_val, pred_val, average = "macro").round(3),
                     "F1 Score Train": f1_score(labels_train, pred_train, average="macro").round(3),
                     "F1 Score Validation": f1_score(labels_val, pred_val, average="macro").round(3)}


In [ ]:
store_eval_score(evaluation_scores, "XGBoost_tuned")

In [ ]:
## Multiclass Confusion Matrix




## Speed improvements through dimensionality reduction

### Checking Feature importance of RF

In [ ]:
# Defining the function to plot the digits of feature importance
# Adapted from Aurelien Geron:

pix_res = 48

def plot_digit(data):
    image = data.reshape(pix_res, pix_res)
    plt.imshow(image, cmap = mpl.cm.hot,
               interpolation="nearest")
    plt.axis("off")

In [ ]:
# Adding feature importances from 3 RGB values to one pixel
feature_imp_sum = np.empty([(int(len(classifier_RF_tuned.feature_importances_)/3)),])

for itr in range(int(len(classifier_RF_tuned.feature_importances_)/3)):
    r = int(itr*3)
    g = int(r+1)
    b = int(g+1)
    feature_imp_sum[itr] = classifier_RF_tuned.feature_importances_[r] + classifier_RF_tuned.feature_importances_[g] + classifier_RF_tuned.feature_importances_[b]


In [ ]:
# Plotting feature importance sum for every pixel to a plot and save it
plot_digit(feature_imp_sum)

cbar = plt.colorbar(ticks=[feature_imp_sum.min(), feature_imp_sum.max()])
cbar.ax.set_yticklabels(['Not important', 'Very important'])

In [ ]:
save_fig("RandomForest_feature_importance_plot_full_data")

### Performing Principal Component Analysis on the training data

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
img_data_full.shape

In [ ]:
# defining to keep 99% of the variance of the data
pca = PCA(.99)
img_data_full_red = pca.fit_transform(img_data_full)

In [ ]:
# checking how many features are left
img_data_full_red.shape

In [ ]:
# Split into training and test set - 10.000 test set / 40.000 full training set
# stratify = labels splits it proportionally to classes in the dataset

from sklearn.model_selection import train_test_split
img_data_train_full, img_data_test, labels_train_full, labels_test, pic_ids_train_full, pic_ids_test = train_test_split(img_data_full_red , labels_full, pic_ids_full, stratify=labels_full, test_size=0.2, random_state=42)

In [ ]:
# Split into training and validation set - 30.000 training set / 10.000 validation set
from sklearn.model_selection import train_test_split
img_data_train, img_data_val, labels_train, labels_val, pic_ids_train, pic_ids_val = train_test_split(img_data_train_full , labels_train_full, pic_ids_train_full, train_size=30000, random_state=42)

### Evaluate speed tuned RF classifier

In [ ]:
# Victor: muss man dann hier nochmal das ganze training (direkt mit den gefundenen hyperparametern) und
# evaluation nochmal machen? Schon oder?
# und dann besteht die Gefahr, dass die Hyperparameter nicht mehr ideal sind und man die dann nochmal
# fine tunen muss (so wars bei Thilo) aber falls die predictions nicht zu schlecht sind können wir auch argumentieren
# dass wir das aufgrund von run-time and scope limitations of our project nicht mehr machen

In [ ]:
## define the tuned classifier with the best hyperparamters
## these here are still random ones

num_trees = 10 # change to actual value

classifier_RF_tuned_PCA = RandomForestClassifier(class_weight='balanced',
                                      criterion='gini',
                                      max_depth=55,
                                      max_features='log2',
                                      min_samples_leaf=0.005,
                                      min_samples_split=0.005,
                                      n_estimators=num_trees)

classifier_RF_tuned_PCA.fit(img_data_train,labels_train)

In [ ]:
# store predictions of classifier
pred_train = classifier_RF_tuned_PCA.predict(img_data_train)
pred_val = classifier_RF_tuned_PCA.predict(img_data_val)

# evaluate classifier and store metrics
evaluation_scores = {"Precision Score Train": precision_score(labels_train, pred_train, average = "macro").round(3),
                     "Precision Score Validation": precision_score(labels_val, pred_val, average = "macro").round(3),
                     "Recall Score Train": recall_score(labels_train, pred_train, average = "macro").round(3),
                     "Recall Score Validation": recall_score(labels_val, pred_val, average = "macro").round(3),
                     "F1 Score Train": f1_score(labels_train, pred_train, average="macro").round(3),
                     "F1 Score Validation": f1_score(labels_val, pred_val, average="macro").round(3)}


In [ ]:
store_eval_score(evaluation_scores, "RandomForest_tuned_PCA")

### Evaluate speed tuned XGB classifier

In [ ]:
## define the tuned classifier with the best hyperparamters
## these here are still random ones

classifier_XGB_tuned_PCA = xgb.XGBClassifier(colsample_bytree=0.7, gamma=0.5, learning_rate=0.3,max_depth=25, subsample=0.5,n_estimators=700)

classifier_XGB_tuned_PCA.fit(img_data_train,labels_train)

In [ ]:
# store predictions of classifier
pred_train = classifier_XGB_tuned_PCA.predict(img_data_train)
pred_val = classifier_XGB_tuned_PCA.predict(img_data_val)

# evaluate classifier and store metrics
evaluation_scores = {"Precision Score Train": precision_score(labels_train, pred_train, average = "macro").round(3),
                     "Precision Score Validation": precision_score(labels_val, pred_val, average = "macro").round(3),
                     "Recall Score Train": recall_score(labels_train, pred_train, average = "macro").round(3),
                     "Recall Score Validation": recall_score(labels_val, pred_val, average = "macro").round(3),
                     "F1 Score Train": f1_score(labels_train, pred_train, average="macro").round(3),
                     "F1 Score Validation": f1_score(labels_val, pred_val, average="macro").round(3)}


In [ ]:
store_eval_score(evaluation_scores, "XGBoost_tuned_PCA")


## Final Evaluation on Test Set

## test baseline RF on test set

In [ ]:
# store predictions of classifier

pred_test = classifier_LR.predict(img_data_test)

In [ ]:
# evaluate classifier and store metrics

evaluation_scores = {"Precision Score Test": precision_score(labels_test, pred_test, average = "macro").round(3),
    "Recall Score Test": recall_score(labels_test, pred_test, average = "macro").round(3),
    "F1 Score Test": f1_score(labels_test, pred_test, average="macro").round(3)}

In [ ]:
# save evaluation scores

def store_eval_score_test(image_df, classifier_name):
    savez_compressed(OUTPUT_PATH_TEST_EVAL + "/evaluation_scores_"+str(classifier_name)+".npz",image_df)
    print("file successfully stored in: output/ml_models/03_test_set_evaluation")

In [ ]:
store_eval_score_test(evaluation_scores,"Logistic Regression")

## test tuned RF on test set

In [ ]:
# store predictions of classifier

pred_test = classifier_RF_tuned.predict(img_data_test)

In [ ]:
# evaluate classifier and store metrics

evaluation_scores = {"Precision Score Test": precision_score(labels_test, pred_test, average = "macro").round(3),
                     "Recall Score Test": recall_score(labels_test, pred_test, average = "macro").round(3),
                     "F1 Score Test": f1_score(labels_test, pred_test, average="macro").round(3)}

In [ ]:
# store evaluation scores

store_eval_score_test(evaluation_scores,"Tuned Random Forest")

### test tuned XGBoost on test set

In [ ]:
# store predictions of classifier

pred_test = classifier_XGB_tuned_PCA.predict(img_data_test)

In [ ]:
# evaluate classifier and store metrics
evaluation_scores = {"Precision Score Test": precision_score(labels_test, pred_test, average = "macro").round(3),
                     "Recall Score Test": recall_score(labels_test, pred_test, average = "macro").round(3),
                     "F1 Score Test": f1_score(labels_test, pred_test, average="macro").round(3)}

In [ ]:
# store evaluation scores

store_eval_score_test(evaluation_scores,"Tuned XGBoost")
